In [1]:
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../../')
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mat73
from scipy.io import loadmat
import os
import time
from search import Search
from utils import Utils, Struct
from individual import Individual
from initialization import Initialization

In [2]:
original_df = pd.read_csv("../../results/results.csv", delimiter=";")
'250' in original_df["instance"][0].split("_")

True

In [3]:
original_df["instance"].unique()

array(['Instance_250_sparse.mat'], dtype=object)

In [29]:
def perform_improvement(path_to_csv, path_to_save, time_to_search, n_individuals, local_search_method = "LSBI", path_relinking_method = "backward"):
    original_df = pd.read_csv(path_to_csv, delimiter=";") # set the path to the results.csv to get chromosomes to perform improvements file here
    df = original_df.copy()
    instance_names = original_df["instance"].unique()
    for i, instance_name in enumerate(instance_names):
        instance = loadmat(os.path.join("../../instances/", instance_name))
        known_result = mat73.loadmat(os.path.join("../../known_results", "x_ls_" + instance_name.replace("Instance_", "")))
        best_known_Z = Search.ldet_objval(instance["A"], known_result["x_ls"].reshape(-1,1))
        df_instance = df[df["instance"] == instance_name]
        df_instance = df_instance[pd.notna(df_instance["improvement_candidates"])]
        print(f"Row Indexes to be processed: {df_instance.index.to_list()}")
        for row in df_instance.itertuples():
            start = time.time()
            A = instance["A"]
            n = row.n
            s = row.s
            encoding = row.encoding_method
            env=Struct({"A": A, "n": n, "s": s, "encoding": encoding})

            #hashes = row.improvement_candidates_hashes.split(",")
            #chromosomes = [Utils.hash_2_chromosome(h) for h in hashes][:n_individuals]
            raw_chromosomes =  row.improvement_candidates.split(",")
            stop = False
            msg = ""
            for c in raw_chromosomes:
                if np.array(list(c)).shape[0] != n:
                    msg = f"Error: chromosome shape is not correct at row Index: {row.Index} - c_shape: {np.array(list(c)).shape[0]} - n: {n}"
                    stop = True
                    break
                if np.unique(c) == '#':
                    stop = True
                    msg = f"Error: chromosome is not correct at row Index: {row.Index} - c: {c}"
                    break
            if stop:
                print(msg)
                continue
            chromosomes = [np.array(list(c), dtype=int).reshape(-1,1) for c in raw_chromosomes][:n_individuals]
            individuals = [Individual(chromosome=c, environment=env) for c in chromosomes]

            # Local Search
            ls_individuals, ls_solutions = Search.local_search(environment=env, individuals = individuals, max_time = time_to_search, best_sol = best_known_Z, local_search_method = local_search_method)
            df_instance.at[row.Index, "ls_improved_solution"] = max(ls_solutions) > row.best_solution_found
            df_instance.at[row.Index, "ls_best_sol"] = max(ls_solutions)
            df_instance.at[row.Index, "ls_best_sol_gap"] = (max(ls_solutions) - best_known_Z)/abs(best_known_Z)
            df_instance.at[row.Index, "ls_total_time"] = time_to_search
            df_instance.at[row.Index, "ls_sol_values"] = ",".join([str(sol) for sol in ls_solutions])
            df_instance.at[row.Index, "ls_individuals_hashes"] = ",".join([str(ind.individual_hash) for ind in ls_individuals])
            df_instance.at[row.Index, "local_search_method"] = local_search_method

            # Path Relinking
            pr_hashes, pr_solutions, pr_time = Search.path_relinking(chromosomes, A, path_relinking_method)
            df_instance.at[row.Index, "pr_best_sol"] = max(pr_solutions)
            df_instance.at[row.Index, "pr_best_sol_gap"] = (max(pr_solutions) - best_known_Z)/abs(best_known_Z)
            df_instance.at[row.Index, "pr_improved_solution"] = max(pr_solutions) > row.best_solution_found
            df_instance.at[row.Index, "pr_individuals_hashes"] = ",".join([str(h) for h in pr_hashes])
            df_instance.at[row.Index, "pr_sol_values"] = ",".join([str(s) for s in pr_solutions])
            df_instance.at[row.Index, "pr_total_time"] = pr_time
            if os.path.exists(path_to_save):
                header = False
            else:
                header = True
            df_instance.to_csv(path_to_save, index=False, sep=";", mode="a", header=header)
            print(f"Row Index: {row.Index} finished in {time.time()-start}.")

path_to_csv = "../../results/results_init_test_improve_incomplete.csv" # set the path to the results.csv to get chromosomes to perform improvements file here
path_to_save = "../../results/improvement_results.csv"
ls_max_time = 120
original_df = pd.read_csv(path_to_csv, delimiter=";")
perform_improvement(path_to_csv, path_to_save, ls_max_time, 5)

Row Indexes to be processed: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
Row Index: 0 finished in 0.06905484199523926.
Row Index: 1 finished in 0.06671810150146484.
Row Index: 2 finished in 0.0657496452331543.
Row Index: 3 finished in 0.06632351875305176.
Row Index: 4 finished in 0.06432461738586426.
Row Index: 5 finished in 0.06156611442565918.
Row Index: 6 finished in 0.06479310989379883.
Row Index: 7 finished in 0.058304548263549805.
Row Index: 8 finished in 0.057405948638916016.
Row Index: 9 finished in 0.11479854583740234.
Row Index: 10 finished in 0.11470437049865723.
Row Index: 11 finished in 0.05803108215332031.
Row Index: 12 finished in 0.0721595287322998.
Row Index: 13 finished in 0.054028987884521484.
Error: chromosome shape is not correct at row Index: 14 - c_shape: 255 - n: 40
Row Index: 15 finished in 0.07784914970397949.
Row Index: 16 finished in 0.09034395217895508.
Row Index: 17 finished in 0.0638978481

In [ ]:
instance = loadmat(os.path.join("../../instances/", "Instance_250_sparse.mat"))
env = Struct({"initialization_method": "random", "A": instance["A"], "n": instance["A"].shape[0], "s": 150, "encoding": "binary"})
population, best_sol = Initialization.initialize_population(env, 2)

In [ ]:
hashes, z_vals = Search.pr_unidirectional(population[0].chromosome, population[1].chromosome, env.A)

In [ ]:
z_vals

In [ ]:
(new_x == 1).sum()

In [ ]:
population[1].chromosome.sum()

In [ ]:
len(new_x)

In [ ]:
Search.ldet_objval(env.A, new_x)